[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/EMQA/blob/main/EMQA_ml_compare/EMQA_ml_compare.ipynb)

# EMQA_ml_compare
Rolling 1-step-ahead ML model comparison: Linear Regression, Random Forest, Gradient Boosting on Brent crude.
**Output:** `ml_compare.pdf`

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

plt.rcParams.update({
    'figure.facecolor': 'none',
    'axes.facecolor': 'none',
    'savefig.facecolor': 'none',
    'savefig.transparent': True,
    'axes.grid': False,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'font.size': 11,
    'figure.figsize': (12, 6),
})

COLORS = {
    'blue': '#1A3A6E', 'red': '#CD0000', 'green': '#2E7D32',
    'orange': '#E67E22', 'purple': '#8E44AD', 'gray': '#808080',
    'cyan': '#00BCD4', 'amber': '#B5853F'
}

def save_fig(fig, name):
    fig.savefig(name, bbox_inches='tight', transparent=True, dpi=300)
    print(f"Saved: {name}")

In [ ]:
import yfinance as yf

def fetch(ticker, start='2020-01-01', end='2025-12-31'):
    d = yf.download(ticker, start=start, end=end, progress=False)
    if isinstance(d.columns, pd.MultiIndex):
        return d['Close'].squeeze().dropna()
    return d['Close'].dropna()

brent = fetch('BZ=F', start='2018-01-01')
df = pd.DataFrame({'price': brent})
df['return'] = np.log(df['price'] / df['price'].shift(1))

# Features: lag1-lag5 returns, rolling vol and mean (5,20)
for lag in range(1, 6):
    df[f'ret_lag_{lag}'] = df['return'].shift(lag)

df['roll_vol_5']  = df['return'].rolling(5).std()
df['roll_vol_20'] = df['return'].rolling(20).std()
df['roll_mean_5']  = df['return'].rolling(5).mean()
df['roll_mean_20'] = df['return'].rolling(20).mean()

# Target: next-day return
df['target'] = df['return'].shift(-1)
df = df.dropna()

feature_cols = [c for c in df.columns if c not in ['price', 'return', 'target']]
print(f"Features: {feature_cols}")
print(f"Total observations: {len(df)}")

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, r2_score

# Rolling expanding-window 1-step-ahead forecast for 3 models
init_train = int(len(df) * 0.7)
retrain_every = 20

model_specs = {
    'Linear Regression': None,
    'Random Forest': None,
    'Gradient Boosting': None,
}

# Storage
preds = {m: [] for m in model_specs}
actuals = []
dates = []

for i in range(init_train, len(df)):
    step = i - init_train

    # Retrain every 20 steps or first iteration
    if step % retrain_every == 0:
        X_tr = df[feature_cols].iloc[:i].values
        y_tr = df['target'].iloc[:i].values

        model_specs['Linear Regression'] = LinearRegression().fit(X_tr, y_tr)
        model_specs['Random Forest'] = RandomForestRegressor(
            n_estimators=200, max_depth=10, random_state=42, n_jobs=-1).fit(X_tr, y_tr)
        model_specs['Gradient Boosting'] = GradientBoostingRegressor(
            n_estimators=200, max_depth=5, random_state=42).fit(X_tr, y_tr)

    X_step = df[feature_cols].iloc[i:i+1].values

    for name, model in model_specs.items():
        preds[name].append(model.predict(X_step)[0])

    actuals.append(df['target'].iloc[i])
    dates.append(df.index[i])

actuals = np.array(actuals)
dates = pd.DatetimeIndex(dates)
for m in preds:
    preds[m] = np.array(preds[m])

# Metrics
results = {}
for name in model_specs:
    y_pred = preds[name]
    mae = mean_absolute_error(actuals, y_pred)
    r2 = r2_score(actuals, y_pred)

    # Directional Sharpe: trade sign of predicted return
    strat_ret = np.sign(y_pred) * actuals
    sharpe = strat_ret.mean() / strat_ret.std() * np.sqrt(252) if strat_ret.std() > 0 else 0

    results[name] = {'MAE': mae, 'R2': r2, 'Sharpe': sharpe}
    print(f"{name:22s}  MAE={mae:.6f}  R2={r2:.4f}  Sharpe={sharpe:.2f}")

res_df = pd.DataFrame(results).T

In [ ]:
# Plot 1: Cumulative returns of each model's strategy vs buy-and-hold
fig, ax = plt.subplots(figsize=(12, 6))

# Buy-and-hold cumulative return
cum_bh = np.cumsum(actuals)
ax.plot(dates, cum_bh, color=COLORS['gray'], lw=1.8, label='Buy & Hold')

model_colors = [COLORS['blue'], COLORS['green'], COLORS['orange']]
for (name, y_pred), col in zip(preds.items(), model_colors):
    strat_ret = np.sign(y_pred) * actuals
    cum_strat = np.cumsum(strat_ret)
    ax.plot(dates, cum_strat, color=col, lw=1.5, label=name)

ax.set_xlabel('Date')
ax.set_ylabel('Cumulative Log Return')
ax.set_title('Rolling 1-Step-Ahead: Cumulative Strategy Returns vs Buy & Hold')
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.10), frameon=False, ncol=4)

plt.tight_layout()
plt.show()

In [ ]:
# Plot 2: Bar chart comparing R2, MAE, Sharpe for each model
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

bar_colors = [COLORS['blue'], COLORS['green'], COLORS['orange']]
metrics = ['MAE', 'R2', 'Sharpe']
titles = ['Mean Absolute Error (lower is better)',
          'R-squared (higher is better)',
          'Sharpe Ratio (higher is better)']
ylabels = ['MAE', '$R^2$', 'Sharpe']

for ax, metric, title, ylabel in zip(axes, metrics, titles, ylabels):
    vals = res_df[metric].values
    bars = ax.bar(res_df.index, vals, color=bar_colors, width=0.5, edgecolor='white')
    for bar, val in zip(bars, vals):
        offset = abs(val) * 0.03 if val != 0 else 0.001
        va = 'bottom' if val >= 0 else 'top'
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + offset,
                f'{val:.4f}', ha='center', va=va, fontsize=10)
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    ax.tick_params(axis='x', rotation=15)
    if metric == 'R2':
        ax.axhline(0, color=COLORS['gray'], linestyle=':', linewidth=0.8)

plt.tight_layout()
save_fig(fig, 'ml_compare.pdf')
plt.show()